# Step 3: Multiple Imputation by Chained Equations (MICE)

In [ ]:
from ml_tools.MICE_imputation import run_mice_pipeline
from paths import DATA_DIR, ENGINEERED_CSVS_DIR, MICE_IMPUTED_DATASETS_DIR, MICE_METRICS_DIR
from helpers.constants import TARGETS
from ml_tools.utilities import deserialize_object
import os

In [ ]:
binary_columns_path = os.path.join(DATA_DIR, "BINARY_COLUMNS_list.joblib")
binary_columns = deserialize_object(filepath=binary_columns_path)

In [ ]:
run_mice_pipeline(df_path_or_dir=ENGINEERED_CSVS_DIR,
                  target_columns=TARGETS,
                  save_datasets_dir=MICE_IMPUTED_DATASETS_DIR,
                  save_metrics_dir=MICE_METRICS_DIR,
                  binary_columns=binary_columns,
                  iterations=30)